Let's try some basic tests of `vbgraph.jl`:

In [1]:
include("vbgraph.jl")

E (generic function with 1 method)

Let's try to make a Gaussian node with several different sizes and parameterizations:

In [2]:
ndims = (5, 5)  # node dimensions
vardims = 3  # dimension of variable

3

In [3]:
μ = Array{Vector{Float64}}(ndims)
Σ = Array{Matrix{Float64}}(ndims)

for idx in eachindex(μ)
    μ[idx] = rand(vardims)
end

for idx in eachindex(Σ)
    C = rand(vardims, vardims)
    Σ[idx] = C * C'
end

In [4]:
μ[1]

3-element Array{Float64,1}:
 0.7137  
 0.93228 
 0.983038

In [5]:
Σ[1]

3x3 Array{Float64,2}:
 1.07128   0.656745  0.309996
 0.656745  0.765685  0.288767
 0.309996  0.288767  0.31453 

In [6]:
det(Σ[2])

0.06398447848039057

In [7]:
pars = (μ, Σ)
aa = map(MultivariateNormal, pars...)

5x5 Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}:
 FullNormal(
dim: 3
μ: [0.7136997033264394,0.9322795320075157,0.9830376060634602]
Σ: 3x3 Array{Float64,2}:
 1.07128   0.656745  0.309996
 0.656745  0.765685  0.288767
 0.309996  0.288767  0.31453 
)
   …  FullNormal(
dim: 3
μ: [0.8653547499344048,0.12793633209141264,0.6176918508539582]
Σ: 3x3 Array{Float64,2}:
 1.09935   0.750438  1.115   
 0.750438  0.513072  0.758835
 1.115     0.758835  1.14877 
)
 
 FullNormal(
dim: 3
μ: [0.2300723257005466,0.012749336997825544,0.9851931269677983]
Σ: 3x3 Array{Float64,2}:
 0.302462  0.36545  0.422055
 0.36545   1.52536  1.2389  
 0.422055  1.2389   1.2744  
)
       FullNormal(
dim: 3
μ: [0.42765819437484365,0.9533440773520776,0.8038423300331985]
Σ: 3x3 Array{Float64,2}:
 1.27197   0.556267  0.620212
 0.556267  0.534958  0.608186
 0.620212  0.608186  0.843594
)
 
 FullNormal(
dim: 3
μ: [0.1531479416081567,0.24493876254222813,0.4184946506977618]
Σ: 3x3 Array{Float64,2}:
 1.19231   0

In [8]:
rr = RandomNode(aa, Factor[]);

In [9]:
typeof(aa)

Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}

In [10]:
typeof(aa) <: AbstractArray{typeof(aa[1]), 2}

true

In [11]:
ss = RandomNode(MultivariateNormal, pars...);

In [12]:
E(ss)

5x5 Array{Array{Float64,1},2}:
 [0.7136997033264394,0.9322795320075157,0.9830376060634602]    …  [0.8653547499344048,0.12793633209141264,0.6176918508539582] 
 [0.2300723257005466,0.012749336997825544,0.9851931269677983]     [0.42765819437484365,0.9533440773520776,0.8038423300331985] 
 [0.1531479416081567,0.24493876254222813,0.4184946506977618]      [0.7947241660126982,0.006077981979288882,0.4928546861223293]
 [0.21154329808457395,0.45656504166272893,0.916757339552382]      [0.9057419562397029,0.23337930905661297,0.7055802078211781] 
 [0.46005731205173417,0.4740865938361418,0.7623417501061427]      [0.9607337568522356,0.9179510075830799,0.45287409013420166] 

In [14]:
methods(E)

# 1 method for generic function "E":
E(x::RandomNode{D<:Distributions.Distribution{F<:Distributions.VariateForm,S<:Distributions.ValueSupport},N}) at /Users/jmxp/code/vbgraph/vbgraph.jl:26